In [45]:
import pandas as pd
import spacy
import numpy as np
import pyLDAvis.gensim_models
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from bs4 import BeautifulSoup

In [2]:
df_articles = pd.read_csv('sävsjö_articles.csv')
df_articles.head()
df_articles = df_articles.dropna()
df_articles.head()

,ID,Title,Text
0,2,Aktiespararna säger ja till Gunvorbudet<span> ...,Aktiespararna rekommenderar sina medlemmar att...
2,7,Man fast i en timme i grop<span> - kuriren.nu<...,En man i 60-årsåldern föll ner i en grop som v...
3,8,Mannen fastnade i en grop  i flera timmar<spa...,En man i 60-årsåldern trillade ner i ett grävt...
4,56,En motorcykel,Här är den längre versionen.
5,66,Stulna braskaminer för 50 000 ska värma,Det börjar bli vinter och kallt. Då är det skö...


In [68]:
for col in df_articles.columns[1:]:
    df_articles[col] = df_articles[col].astype(str)
    df_articles[col] = df_articles[col].apply(lambda x: BeautifulSoup(x).get_text())
df_articles = df_articles.replace(r'[^\w\s]|_', '', regex=True)

c:\Users\emilk\miniconda3\envs\tf-gpu\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [5]:
nlp = spacy.load("sv_core_news_sm")
removal= ['ADV','PRON','CCONJ','PUNCT','PART','DET','ADP','SPACE', 'NUM', 'SYM']
tokens = []
for summary in nlp.pipe(df_articles['Text']):
   proj_tok = [token.lemma_.lower() for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
   tokens.append(proj_tok)

In [6]:
df_copy = df_articles.copy()
df_copy['tokens'] = tokens
df_copy['tokens'] 

0       [aktiespararna, rekommendera, medlem, anta, bu...
2       [föll, grop, grävd, egen, tomt, sävsjö, timme,...
3          [trilla, grävt, hål, tomt, grop, stund, hitta]
4                                         [vara, version]
5       [börja, vinter, vara, skön, braskamin, uppenba...
                              ...                        
8600                                               [ewry]
8601    [ljus, glödorange, sol, sänka, horison, onsdag...
8602    [folkdansarna, sunne, hagfors, fredag, folkdan...
8603    [text, vara, insändare, åsikt, text, vara, skr...
8604    [soluppgång, krispig, morgondimman, stå, anett...
Name: tokens, Length: 8603, dtype: object

In [8]:
dict = Dictionary(df_copy['tokens'])
dict.token2id

{'aktiespararna': 0,
 'anse': 1,
 'anta': 2,
 'budpliktsbud': 3,
 'group': 4,
 'gunvor': 5,
 'lägga': 6,
 'medlem': 7,
 'rekommendera': 8,
 'rörvik': 9,
 'timber': 10,
 'djup': 11,
 'egen': 12,
 'föll': 13,
 'grop': 14,
 'grävd': 15,
 'hål': 16,
 'ligga': 17,
 'meter': 18,
 'sävsjö': 19,
 'timme': 20,
 'tomt': 21,
 'grävt': 22,
 'hitta': 23,
 'stund': 24,
 'trilla': 25,
 'vara': 26,
 'version': 27,
 'braskamin': 28,
 'börja': 29,
 'förråd': 30,
 'försvann': 31,
 'helg': 32,
 'kamin': 33,
 'klippa': 34,
 'krona': 35,
 'skön': 36,
 'smålandsvillan': 37,
 'stängsl': 38,
 'uppenbarligen': 39,
 'vinter': 40,
 'visste': 41,
 'vrigstad': 42,
 'värde': 43,
 'arbetsgivare': 44,
 'basa': 45,
 'claesson': 46,
 'elev': 47,
 'förhandlingsfråga': 48,
 'heta': 49,
 'hittillsvarande': 50,
 'jobba': 51,
 'jönköping': 52,
 'klar': 53,
 'kommun': 54,
 'läre': 55,
 'ny': 56,
 'skolchef': 57,
 'stefan': 58,
 'beröra': 59,
 'drabba': 60,
 'fjärrtrafik': 61,
 'fredagsmorgon': 62,
 'kontakt': 63,
 'krösatåg':

In [52]:
dict.filter_extremes(no_below=3, no_above=0.6, keep_n=500)

In [53]:
dict.token2id

{'anse': 0,
 'lägga': 1,
 'rörvik': 2,
 'egen': 3,
 'sävsjö': 4,
 'timme': 5,
 'hitta': 6,
 'börja': 7,
 'krona': 8,
 'vrigstad': 9,
 'elev': 10,
 'jobba': 11,
 'jönköping': 12,
 'klar': 13,
 'kommun': 14,
 'ny': 15,
 'stefan': 16,
 'kontakt': 17,
 'ta': 18,
 'tid': 19,
 'förslag': 20,
 'gammal': 21,
 'grupp': 22,
 'lång': 23,
 'plan': 24,
 'välja': 25,
 'ge': 26,
 'personal': 27,
 'vecka': 28,
 'bil': 29,
 'köra': 30,
 'polis': 31,
 'se': 32,
 'område': 33,
 'företag': 34,
 'handla': 35,
 'kvinna': 36,
 'avsluta': 37,
 'besök': 38,
 'fortsätta': 39,
 'slut': 40,
 'år': 41,
 'dags': 42,
 'extra': 43,
 'form': 44,
 'hålla': 45,
 'klocka': 46,
 'ske': 47,
 'söndag': 48,
 'dra': 49,
 'fredag': 50,
 'kväll': 51,
 'lokal': 52,
 's': 53,
 'stockaryd': 54,
 'barn': 55,
 'lördag': 56,
 'berätta': 57,
 'framtid': 58,
 'kännas': 59,
 'mål': 60,
 'samband': 61,
 'stå': 62,
 'vänta': 63,
 'natt': 64,
 'innebära': 65,
 'visa': 66,
 'tycka': 67,
 'väg': 68,
 'antal': 69,
 'bruka': 70,
 'fall': 71,
 

In [39]:
bagOfWords = [dict.doc2bow(i) for i in df_copy['tokens']]

In [40]:
LDA_Model = LdaMulticore(corpus=bagOfWords, id2word=dict, iterations=50, num_topics=5, workers = 4, passes=10)

In [41]:
LDA_Model.print_topics(-1)

[(0,
  '0.070*"år" + 0.048*"barn" + 0.046*"sävsjö" + 0.029*"ta" + 0.020*"ny" + 0.018*"vrigstad" + 0.018*"se" + 0.017*"berätta" + 0.015*"hoppas" + 0.014*"kännas"'),
 (1,
  '0.050*"år" + 0.046*"kommun" + 0.042*"län" + 0.038*"person" + 0.031*"procent" + 0.030*"öka" + 0.025*"sävsjö" + 0.024*"sverige" + 0.024*"fortsätta" + 0.023*"ta"'),
 (2,
  '0.031*"berätta" + 0.030*"ta" + 0.027*"se" + 0.018*"ny" + 0.017*"hel" + 0.016*"själv" + 0.016*"stå" + 0.016*"tid" + 0.016*"sävsjö" + 0.015*"gång"'),
 (3,
  '0.070*"elev" + 0.068*"skola" + 0.067*"polis" + 0.043*"bil" + 0.042*"sävsjö" + 0.040*"plats" + 0.035*"person" + 0.021*"man" + 0.020*"väg" + 0.019*"räddningstjänst"'),
 (4,
  '0.100*"kommun" + 0.063*"sävsjö" + 0.033*"ta" + 0.029*"ny" + 0.029*"stefan" + 0.027*"gustafsson" + 0.025*"se" + 0.023*"ordförande" + 0.021*"kommunstyrels" + 0.019*"förening"')]

In [42]:
df_copy['topic'] = [sorted(LDA_Model[bagOfWords][text])[0][0] for text in range(len(df_copy['Text']))]

In [43]:
df_copy['Text'][0]

'Aktiespararna rekommenderar sina medlemmar att anta budpliktsbudet  som Gunvor Group har lagt på Rörvik Timber. Aktiespararna anser...'

In [44]:
LDA_Model[bagOfWords][0]

[(0, 0.0503211),
 (1, 0.050244693),
 (2, 0.05042992),
 (3, 0.050713573),
 (4, 0.7982907)]

In [46]:
display = pyLDAvis.gensim_models.prepare(LDA_Model, bagOfWords, dict)
pyLDAvis.display(display)

c:\Users\emilk\miniconda3\envs\tf-gpu\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
